<span style="color: #2874A6;  font-weight: bold;">Unsloth Library Installation and Setup</span>

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

<span style="color: #2874A6; font-weight: bold;">Loading the LLAMA 3.1 8B Instruct Model and quantizing it to 4 bits for faster inference.</span> <br>


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

<span style="color:#2874A6 ; font-weight: bold;">Checking the layers of model</span>

model

<span style="color: #2874A6; font-weight: bold;">Applying Parameter-Efficient Fine-Tuning with PEFT to only the linear layers of the model</span>

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

<span style="color: #2874A6; font-weight: bold;">This code prepares and formats a dataset for training or fine-tuning a language model and converts the model to the Alpaca Prompt Template.</span>

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["context"]
    inputs       = examples["question"]
    outputs      = examples["answer"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

<span style="color: #2874A6; font-weight: bold;">We are also downloading the model which we uploaded to HuggingFace.</span>

In [ ]:
from datasets import load_dataset
dataset = load_dataset("visharxd/medical-qna", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

<span style="color: #2874A6; font-weight: bold;">This code sets up a fine-tuning process for a language model using the SFTTrainer from the trl library.</span>

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 700,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

<span style="color: #2874A6; font-weight: bold;"> Triggers the training process for the model using the settings and dataset specified in the SFTTrainer configuration.</span>

In [ ]:
trainer_stats = trainer.train()

<span style="color: #2874A6; font-weight: bold;"> Pushes the fine-tuned model to the Hugging Face Model Hub under a specific repository.</span>

model.push_to_hub_gguf("visharxd/llama3-finetuned-heathdata/", tokenizer, quantization_method = "q4_k_m", token = YOUR_HUGGINGFACE_API_KEY)
